<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/78_voronoi_polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [ ]:
%%capture
!pip install geovoronoi mapclassify

In [ ]:
import geopandas as gpd
import pooch
from geovoronoi import voronoi_regions_from_coords
from shapely import Point

In [ ]:
# @title Datasets

def get_subway_restaurants_df():
    return gpd.GeoDataFrame(
        ['1050 Douglas Ave', '1055 Prospect St', '1381 Regent Street',
        '245 Bishop Drive', '265 Main St', '349 King St', '67 Dineen Drive'],
        columns=['address'],
        geometry=[
            Point(-66.651810145219, 45.9893545834717),
            Point(-66.666540802893, 45.9418966781184),
            Point(-66.663865374058, 45.9345242668026),
            Point(-66.684656990680, 45.9464861180492),
            Point(-66.648679050274, 45.9770292880759),
            Point(-66.644999688188, 45.9626679969464),
            Point(-66.645178942774, 45.9445832722727),
        ], crs='EPSG:4326'
    )

aoi_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/main/vector/City_Boundary.zip',
    known_hash='431710489ce01850be3ed0ac816305b41cbd871b14fae7d3c94bb8da469c6aa6'
)

In [ ]:
fredericton = gpd.read_file(aoi_file).to_crs('EPSG:2953')
subway_restaurants = get_subway_restaurants_df().to_crs(fredericton.crs)

In [ ]:
voronoi_polys, _ = voronoi_regions_from_coords(
    subway_restaurants.geometry.values,
    fredericton.geometry.squeeze()
)

voronoi_df = gpd.GeoDataFrame(
    geometry=list(voronoi_polys.values()),
    crs=fredericton.crs
).assign(area_km2=lambda df_: (df_.area / 1_000_000).round(2),
         zone=lambda df_: df_.index + 1)

In [ ]:
m = voronoi_df.explore(width=700, height=600, column='zone', cmap='Set1', legend=False)
subway_restaurants.explore(m=m, color='black')